In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data=pd.read_csv('bank.csv',sep=';')

In [3]:
data.head(10)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787.0,no,no,cellular,19.0,oct,79.0,1.0,-1.0,0.0,unknown,no
1,33,services,married,secondary,no,4789.0,yes,yes,cellular,11.0,may,220.0,1.0,339.0,4.0,failure,no
2,35,management,single,tertiary,no,1350.0,yes,no,cellular,16.0,apr,185.0,1.0,330.0,1.0,failure,no
3,30,management,married,tertiary,no,1476.0,yes,yes,unknown,3.0,jun,199.0,4.0,-1.0,0.0,unknown,no
4,59,blue-collar,married,secondary,no,0.0,yes,no,unknown,5.0,may,226.0,1.0,-1.0,0.0,unknown,no
5,35,management,single,tertiary,no,747.0,no,no,cellular,23.0,feb,141.0,2.0,176.0,3.0,failure,no
6,36,self-employed,married,tertiary,no,307.0,yes,no,cellular,14.0,may,341.0,1.0,330.0,2.0,other,no
7,39,technician,married,secondary,no,147.0,yes,no,cellular,6.0,may,151.0,2.0,-1.0,0.0,unknown,no
8,41,entrepreneur,married,tertiary,no,221.0,yes,no,unknown,14.0,may,57.0,2.0,-1.0,0.0,unknown,no
9,43,services,married,primary,no,-88.0,yes,yes,cellular,17.0,apr,313.0,1.0,147.0,2.0,failure,no


In [4]:
data['y'].replace(['no','yes'],[0,1],inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3498 entries, 0 to 3497
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        3498 non-null   int64  
 1   job        3497 non-null   object 
 2   marital    3497 non-null   object 
 3   education  3497 non-null   object 
 4   default    3497 non-null   object 
 5   balance    3497 non-null   float64
 6   housing    3497 non-null   object 
 7   loan       3497 non-null   object 
 8   contact    3497 non-null   object 
 9   day        3497 non-null   float64
 10  month      3497 non-null   object 
 11  duration   3497 non-null   float64
 12  campaign   3497 non-null   float64
 13  pdays      3497 non-null   float64
 14  previous   3497 non-null   float64
 15  poutcome   3497 non-null   object 
 16  y          3497 non-null   float64
dtypes: float64(7), int64(1), object(9)
memory usage: 464.7+ KB


In [6]:
data.dropna(inplace=True)

In [7]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [8]:
X=pd.get_dummies(X)

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
X_train.shape

(2797, 51)

In [11]:
y_train.shape

(2797,)

In [12]:
from numpy import exp, array, random, dot


class NeuralNetwork():
    def __init__(self):
        # Seed the random number generator, so it generates the same numbers
        # every time the program runs.
        random.seed(1)

        # We model a single neuron, with 3 input connections and 1 output connection.
        # We assign random weights to a 3 x 1 matrix, with values in the range -1 to 1
        # and mean 0.
        self.synaptic_weights = 2 * random.random((51,1)) - 1

    # The Sigmoid function, which describes an S shaped curve.
    # We pass the weighted sum of the inputs through this function to
    # normalise them between 0 and 1.
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative of the Sigmoid function.
    # This is the gradient of the Sigmoid curve.
    # It indicates how confident we are about the existing weight.
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # We train the neural network through a process of trial and error.
    # Adjusting the synaptic weights each time.
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            # Pass the training set through our neural network (a single neuron).
            output = self.think(training_set_inputs)
            
            # Calculate the error (The difference between the desired output
            # and the predicted output).
            error = training_set_outputs - output
            
            # Multiply the error by the input and again by the gradient of the Sigmoid curve.
            # This means less confident weights are adjusted more.
            # This means inputs, which are zero, do not cause changes to the weights.
            adjustment = dot(training_set_inputs.T, error*self.__sigmoid_derivative(output))
            
            # Adjust the weights.
            self.synaptic_weights += adjustment

    # The neural network thinks.
    def think(self, inputs):
        # Pass inputs through our neural network (our single neuron).
        return self.__sigmoid(dot(inputs, self.synaptic_weights))


if __name__ == "__main__":

    #Intialise a single neuron neural network.
    neural_network = NeuralNetwork()

    print("Random starting synaptic weights: ")
    print(neural_network.synaptic_weights)

    # The training set. We have 4 examples, each consisting of 3 input values
    # and 1 output value.
    training_set_inputs = X_train.to_numpy()
    training_set_outputs = y_train.to_numpy().reshape(((2797, 1)))
    # Train the neural network using a training set.
    # Do it 10,000 times and make small adjustments each time.
    neural_network.train(training_set_inputs, training_set_outputs, 10000)

    print("New synaptic weights after training: ")
    print(neural_network.synaptic_weights)

    # Test the neural network with a new situation.
    print("Considering test dataset ")
    print(neural_network.think(X_test.to_numpy()))

Random starting synaptic weights: 
[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]
 [-0.39533485]
 [-0.70648822]
 [-0.81532281]
 [-0.62747958]
 [-0.30887855]
 [-0.20646505]
 [ 0.07763347]
 [-0.16161097]
 [ 0.370439  ]
 [-0.5910955 ]
 [ 0.75623487]
 [-0.94522481]
 [ 0.34093502]
 [-0.1653904 ]
 [ 0.11737966]
 [-0.71922612]
 [-0.60379702]
 [ 0.60148914]
 [ 0.93652315]
 [-0.37315164]
 [ 0.38464523]
 [ 0.7527783 ]
 [ 0.78921333]
 [-0.82991158]
 [-0.92189043]
 [-0.66033916]
 [ 0.75628501]
 [-0.80330633]
 [-0.15778475]
 [ 0.91577906]
 [ 0.06633057]
 [ 0.38375423]
 [-0.36896874]
 [ 0.37300186]
 [ 0.66925134]
 [-0.96342345]
 [ 0.50028863]
 [ 0.97772218]
 [ 0.49633131]
 [-0.43911202]
 [ 0.57855866]
 [-0.79354799]
 [-0.10421295]
 [ 0.81719101]
 [-0.4127717 ]
 [-0.42444932]
 [-0.73994286]
 [-0.96126608]]
New synaptic weights after training: 
[[-5.00989727e+01]
 [-3.04019810e+02]
 [-2.01297839e+01]
 [-2.37245441e+02]
 [-4.46126084e+00]
 [-1.04932454e+01]
 [-7.16670034e-01]
 [-3.91796257e-01]
 [-3.8121

In [13]:
y_pred=neural_network.think(X_test.to_numpy())

In [14]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8485714285714285
